In [11]:
from functions import *
import pandas as pd
import seaborn as sns
import numpy as np
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
import re
import ppscore as pps

In [17]:
df_training = pd.read_csv("training_data.csv")
df_test = pd.read_csv("forecast_starting_data.csv")
df_macro = pd.read_csv("macro_data.csv")

C:\Users\devas\AppData\Local\Temp\ipykernel_16068\998706994.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_training = pd.read_csv("training_data.csv")


# Initial data cleaning and sample creation
To do some preliminary cleaning of the data, we dropped columns with only NaN values. Then, we created our sample dataset that we used to identify relationships and train the model. 50% of this sample contained all the accounts that were charged off in the dataset (15698 values), while the other 50% of this sample contained a random sample of 15698 accounts that weren't charged off in the dataset, for a total of 31396 accounts.

In [18]:
df_training.drop('charge_off_reason_code', axis=1, inplace=True)
df_training.drop('writeoff_date', axis =1, inplace=True)
# dataless_forecast_cols = []
# for column in df_test.columns:
#     if ((len(df_test[column].unique()) == 1) & len(column)):
#         df_test.append(column)
# df_training1.drop(dataless_forecast_cols, axis=1, inplace=True)

In [19]:
df_no_charge_sample = df_training[df_training['charge_off'] == 0].sample(n=15698)
df_charge = df_training[df_training['charge_off'] == 1]
df_tot_sample = pd.concat([df_charge, df_no_charge_sample], axis=0)

In [20]:
data = []
for column in df_tot_sample.columns:
    data.append(pps.score(df_tot_sample, x=column, y="charge_off"))
df_random_pps1 = pd.DataFrame(data)
df_random_pps1 = df_random_pps1.sort_values(by='ppscore', ascending=False).head(n=20)
df_random_pps1

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
21,charge_off,charge_off,1.000000,predict_itself,True,None,0.000,1.000000,None
26,total_writeoff_amt,charge_off,1.000000,regression,True,mean absolute error,0.488,0.000000,DecisionTreeRegressor()
20,active,charge_off,1.000000,regression,True,mean absolute error,0.488,0.000000,DecisionTreeRegressor()
49,writeoff_type_null,charge_off,0.992651,regression,True,mean absolute error,0.488,0.003586,DecisionTreeRegressor()
25,principal_amt_chrg_off,charge_off,0.970077,regression,True,mean absolute error,0.488,0.014602,DecisionTreeRegressor()
27,fee_chg_off_reversal_amt,charge_off,0.873271,regression,True,mean absolute error,0.488,0.061844,DecisionTreeRegressor()
34,fc_reversals,charge_off,0.850726,regression,True,mean absolute error,0.488,0.072846,DecisionTreeRegressor()
22,charge_off_aged,charge_off,0.802494,regression,True,mean absolute error,0.488,0.096383,DecisionTreeRegressor()
35,fee_reversals,charge_off,0.797362,regression,True,mean absolute error,0.488,0.098888,DecisionTreeRegressor()
17,credit_limit_amt,charge_off,0.679160,regression,True,mean absolute error,0.488,0.156570,DecisionTreeRegressor()


# Feature engineering
We incorporate the macro data into our DataFrame to take economic data into cosideration when making our prediction and constructing our model. Then, we did feature engineering to find the survival times, assign each month a number ranging from 1 to 24, sort the data values, and remove multiple occurences of same observations so that only the observations in which people are charged off or in which the last observations of the people which don't get charged off are remaining. This prepares the data to be applied to the Cox Proportional Hazards Model.


In [22]:
macro_final = pd.DataFrame()
for i, row in df_macro.iterrows():
    if re.search(r"2017$", str(row["Mnemonic"])) or re.search(r"2018$", str(row["Mnemonic"])) or re.search(r"2019$", str(row["Mnemonic"])) or re.search(r"2020$", str(row["Mnemonic"])) or re.search(r"2021$", str(row["Mnemonic"])):
        t1 = pd.DataFrame(row).T
        month_code = format_date(t1['Mnemonic'].iloc[0])
        t1['mth'] = month_code
        macro_final = pd.concat([macro_final, t1], axis=0)

In [23]:
x = pd.DataFrame()
final = pd.DataFrame()
a = 0

for i in df_tot_sample['mth_code'].unique():
    left_block = df_tot_sample[df_tot_sample['mth_code'] == i]
    temp2 = macro_final[macro_final['mth'] == str(i)]
    right_block = pd.concat([temp2] * len(left_block))
    
    left_block = left_block.reset_index(drop=True)
    right_block = right_block.reset_index(drop=True)
    
    x = pd.concat([left_block, right_block], axis=1).reset_index()
    if a == 0:
        final = x
        a += 1
    else:
        final = pd.concat([x, final], axis=0)

In [24]:
final = add_survival_time(final)
final = add_month_nums(final)
final = sort_by_mth_and_snapshot(final)
final = get_uniques(final)
final = final.reset_index()
final.drop('level_0', axis=1, inplace=True)
final.drop('index', axis=1, inplace=True)

# Finding ppscore metrics
In traditional data analysis, the first step to determine any underlying relationships is usually finding the correlation matrix of a sample dataframe. However, the correlation matrix has some limitations, mainly the inability to account for non-linear relationships. Predictive power score (PPS) is an alternative to correlation matrix that has the ability to account for non-linear relationships in its scoring. More documentation and explanation can be found here: https://medium.com/geekculture/predictive-power-score-implementation-in-python-70558bf91f45.
    
We looked at both correlation and ppscore matrices with relation to their ability to predict whether an account will be charged off. From these results, we saw that ___ columns hasdlfkjsa;ldkfja;sldkfj

In [28]:
data = []
for column in final.columns:
    data.append(pps.score(final, x=column, y="charge_off"))

df_random_pps = pd.DataFrame(data)
df_random_pps = df_random_pps.sort_values(by='ppscore', ascending=False).head(n=20)

# Cox Proportional Hazard Model
The Cox Proportional Hazard Model is a parametric model in survival analysis that takes into account multiple variables to analyze their relationship to their survival distribution.


In [ ]:
df_clean_train_final = ...
columns = ...
dfcox = df_clean_train_final[columns]
coxph = CoxPHFitter()
coxph.fit(dfcox, duration_col='time', event_col='charge_off')
coxph.print_summary()

In [ ]:
coxph.plot()

In [ ]:
df_ones = dfcox.loc[dfcox['charge_off'] == 1]
df_after = df_ones['time']
coxph.predict_survival_function(df_ones, conditional_after=df_after)


In [ ]:
coxph.check_assumptions(dfcox)


In [29]:
def findCol(col):
    count = 0
    for column in s.columns:
        if column == col:
            return count
        count += 1

In [30]:
predictors = []
rows_to_predict = dfcox.iloc[predictors, []]
rows_to_predict

NameError: name 'dfcox' is not defined

In [ ]:
survival_function = coxph.predict_survival_function(rows2predict).plot()
survival_function

In [ ]:
start = 
end = 
num_charge_offs = survival_function.loc[start:end, :].diff().iloc[-1].sum() * -1
num_charge_offs